In [2]:
#Previsão de demanda do item 1

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

df = pd.read_csv("plan/train.csv")

df = df[df["item"] == 1]

df['date'] = pd.to_datetime(df['date'])

df = df.filter(["date", "sales"]).groupby([pd.Grouper(key="date", freq="W-MON")]).sum().reset_index()

df

,date,sales
0,2013-01-07,894
1,2013-01-14,863
2,2013-01-21,867
3,2013-01-28,816
4,2013-02-04,969
...,...,...
256,2017-12-04,1490
257,2017-12-11,1311
258,2017-12-18,1308
259,2017-12-25,1262


In [3]:
# Criar a coluna 'shift_sale' com os valores deslocados

df['shift_sale'] = df['sales'].shift(1)

df = df.iloc[1:]

df

,date,sales,shift_sale
1,2013-01-14,863,894.0
2,2013-01-21,867,863.0
3,2013-01-28,816,867.0
4,2013-02-04,969,816.0
5,2013-02-11,920,969.0
...,...,...,...
256,2017-12-04,1490,1818.0
257,2017-12-11,1311,1490.0
258,2017-12-18,1308,1311.0
259,2017-12-25,1262,1308.0


In [4]:
x = df[['sales', 'shift_sale']]
y = df[['sales']]

# Normalização dos dados
scaler = StandardScaler()
x = scaler.fit_transform(x)
y = np.ravel(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

# Criar e treinar o modelo de Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(x_train, y_train)


y_pred = model.predict(x_test)


In [5]:
print("Erro Médio Quadrático (MSE):", mean_squared_error(y_test, y_pred))
print("Erro Médio Absoluto (MAE):", mean_absolute_error(y_test, y_pred))
print("R-squared (R2):", r2_score(y_test, y_pred))
print(y_pred[0])

Erro Médio Quadrático (MSE): 29.530153846153667
Erro Médio Absoluto (MAE): 3.8134615384615356
R-squared (R2): 0.9997456608436666
2137.79
